In [68]:
import numpy as np
import copy

In [69]:
#FILE PARSER: CREATE LIST OF CONNECTIONS
def file_parser(filename):
    newfile = open(filename)

    eof = False
    connect_list = []

    while eof == False:
        entry = newfile.readline()
        entry = entry.rstrip("\n")
        if entry == "END OF INPUT":
            eof = True
        else:
            entry_fix = entry.split(" ")
            connect_list.append(entry_fix)
    return connect_list

In [70]:
def generate_map(city_list):
    unique_cities = {}
    index = 0
    city_map = []
    
    for group in city_list:
        start, end, distance = group
        city_group_start = [start, [[end,int(distance)]]]
        city_group_end = [end, [[start,int(distance)]]]
        if start not in list(unique_cities.keys()):
            unique_cities[start] = index
            index += 1
            city_map.append(city_group_start)
        else:
            city_map[unique_cities[start]][1].append([end, int(distance)])
            
        if end not in list(unique_cities.keys()):
            unique_cities[end] = index
            index += 1
            city_map.append(city_group_end)
        else:
            city_map[unique_cities[end]][1].append([start, int(distance)])
            
        
    return city_map, unique_cities
            

In [71]:
visit = [] # List for visited nodes.
queue = [] 

def bfs(visit, graph, node, city_list): #function for BFS
    
    visit.append(node)
    queue.append(node)

    while queue:          # Creating loop to visit each node
        #print("popped!")
        node = queue.pop(0) 
        #print(m) 

        for neighbor in graph[city_list[node]][1]:
            #print(neighbor)
            if neighbor not in visit:
                visit.append(neighbor)
                queue.append(neighbor[0])
                #print("visited: " + str(visit))
                #print("queue: " + str(queue))

    return visit   # function calling

In [77]:
def finder(start, end, curr_cost, map_list, small_map, fringe, expand, layer, save):
    if start == end:
        final_list = []
        expand.append([start, layer])
        for node in expand:
            final_list.append(node[0])
        return final_list
    else:
        layer += 1
        expand.append([start, layer])
        fringe_copy = small_map[map_list[start]][1]
        
        
        print("-------------STATISTICS-------------")
        print("STARTING NODE: " + start)
        print("CURRENT COST: " + str(curr_cost))
        print("EXPANDED NODES: " + str(expand))
        print("CURRENT FRINGE: " + str(fringe))
        print("NEW FRINGE: " + str(fringe_copy))
        print("CURRENT LAYER: " + str(layer))
        print("------------------------------------")
        
        
        found_goal = False
        
        for i in fringe_copy:
            #print("next up on The Fringe, we have " + str(i[0]) + "!")
            if (i[0] not in (name[0] for name in fringe)):  #looks for duplicates
                city_cost = i[1] + curr_cost
                i.append(layer)
                layer += 1
                #print("New fringe node added: " + str(i))
                fringe.append([i[0], city_cost])
            else:
                #print("there's a copy of " + str(i[0]) + ", we didn't add it")
                #FIX THE DAMN COST
                for index in range(len(fringe)):
                    if fringe[index][0] == i[0]:
                        future_cost = i[1] + curr_cost
                        if future_cost < fringe[index][1]:
                            fringe.remove(fringe[index])
                            i.append(layer)
                            layer += 1
                            fringe.append(i)
                

        fringe.sort(key=lambda x: (x[1], x[2]))
        #print("fringe values")
        #print(fringe)
        
        #IT WORKS UP TO HERE
        #i think we can just pick the first element of the queue?? 
        
        #CHECK FOR GOAL STATE
        for node in fringe:
            if node[0] == end:
                found_goal = True
                final_cost = node[1]
                break
               
        if found_goal == True:
            next_node = end
            new_cost = curr_cost + node[1]
            
        else:
            next_node = fringe[0][0]
            new_cost = fringe[0][1]
            #print("We are advancing with node " + next_node + " and cost " + str(new_cost))
            node_layer = fringe[0][2]
            #print("Node layer for " + str(node) + " is " + str(node_layer))
            
            if (new_cost > save[1]):
                #print("CURRENT ROUTE BAD, NOW SWITCHING")
                temp_save = save
                save = [expand, curr_cost]
                expand = temp_save[0]
                curr_cost = temp_save[1]
            elif node_layer <= layer:
                #print("SWITCH OCCURRING")
                bad_list = []
                for node in expand:
                    #print(str(expand) + " is the list we're working with here")
                    #print("is " + str(node[1]) + " bigger than " + str(node_layer) + "?")
                    if node[1] > node_layer:
                        #print("please print yes")
                        #print(str(node) + " should be in the list")
                        bad_list.append(node)
                #save the old progress in case we need to revert
                old_path = expand
                old_cost = curr_cost
                save = [old_path, old_cost]
                #print("old path saved as: " + str(save[0]))
                
                while len(bad_list) != 0:
                    for node in bad_list:
                        #print(str(node) + ' has been removed.')
                        expand.remove(node)
                        #print('we are supposed to be removing nodes here')
                        #print(expand)
                        bad_list.remove(node)
                layer = node_layer
            
            
            fringe.pop(0)
            #print("expand at end of function: " + str(expand))
            #print(" ")
            
        return finder(next_node, end, new_cost, map_list, small_map, fringe, expand, layer, save)

In [73]:
#THE BIG KAHUNA

def find_route(filename, start, end):
    #open the file and properly initialize everything
    city_file = file_parser(filename)
    city_map, unique_cities = generate_map(city_file)
    
    city_map_copy = copy.deepcopy(city_map)
    unique_cities_copy = copy.deepcopy(unique_cities)

    looplist = bfs(visit, city_map, start, unique_cities) 
    #CASE II: input cities are the same
    if start == end:
        #do the right output
        print("distance: 0 km")
        print("route:")
        print(start + " to " + end + ", 0 km")
        return
    elif (start not in list(unique_cities.keys())) or (end not in list(unique_cities.keys())):
        #do the right output
        print("distance: infinity")
        print("route:")
        print("none")
        return
    #run bfs to see if city distance is even possible
    elif (end not in (i[0] for i in looplist)):
        print("distance: infinity")
        print("route:")
        print("none")
        return
    else: #we are now sure the cities are different and a path can be found
        path = finder(start, end, 0, unique_cities, city_map, [], [], 0, [[], 0])
        distance = 0
        roads = []
        #for each [start, end] pair in the given route
        for i in range(len(path)-1):
            #need to find the corresponding route from the given city
            for j in range(len(city_map_copy[unique_cities_copy[path[i]]][1])):
                if city_map_copy[unique_cities_copy[path[i]]][1][j][0] == path[i+1]:
                    road = city_map_copy[unique_cities_copy[path[i]]][1][j][1]
                    distance += road
                    direction = path[i] + " to " + path[i+1] + ", " + str(road) + " km"
                    roads.append(direction)
        print("distance: " + str(distance) + " km")
        print("route:")
        for i in roads:
            print(i)
        return;
        #format the output right

In [78]:
dicty = {}

-------------STATISTICS-------------
STARTING NODE: Kassel
CURRENT COST: 0
EXPANDED NODES: [['Kassel', 1]]
CURRENT FRINGE: []
NEW FRINGE: [['Hannover', 165], ['Frankfurt', 185]]
CURRENT LAYER: 1
------------------------------------


IndexError: list index out of range